In [38]:
!pip install pytorch_transformers

In [39]:
!pip install transformers

In [40]:
!pip install nlp==0.4.0

In [41]:
!pip install datasets

In [42]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
#from transformers import BertTokenizer,BertForSequenceClassification
from nlp import load_dataset
import torch
import numpy as np

In [7]:
!unzip /content/IMDB_Dataset.csv.zip

Archive:  /content/IMDB_Dataset.csv.zip
  inflating: IMDB Dataset.csv        


In [43]:
dataset = load_dataset('csv', data_files ='/content/IMDB Dataset.csv', split='train') 

Using custom data configuration default


In [44]:
type(dataset.features['sentiment'])

nlp.features.Value

In [45]:
def set_1_or_0(example):
    if example['sentiment'] == 'positive':
        example['label'] = 1
    else:
        example['label'] = 0
    return example

In [46]:
#dataset.features['sentiment'].map({'positive': 1, 'negative':0})
dataset = dataset.map(set_1_or_0)

In [47]:
type(dataset)

nlp.arrow_dataset.Dataset

In [48]:
 dataset

Dataset(features: {'label': Value(dtype='int64', id=None), 'review': Value(dtype='string', id=None), 'sentiment': Value(dtype='string', id=None)}, num_rows: 50000)

In [49]:
dataset = dataset.train_test_split(test_size=0.3)

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

In [50]:
print(dataset)

{'train': Dataset(features: {'label': Value(dtype='int64', id=None), 'review': Value(dtype='string', id=None), 'sentiment': Value(dtype='string', id=None)}, num_rows: 35000), 'test': Dataset(features: {'label': Value(dtype='int64', id=None), 'review': Value(dtype='string', id=None), 'sentiment': Value(dtype='string', id=None)}, num_rows: 15000)}


In [51]:
train_set = dataset['train']
test_set = dataset['test']

In [52]:
print(train_set)
print(test_set)

Dataset(features: {'label': Value(dtype='int64', id=None), 'review': Value(dtype='string', id=None), 'sentiment': Value(dtype='string', id=None)}, num_rows: 35000)
Dataset(features: {'label': Value(dtype='int64', id=None), 'review': Value(dtype='string', id=None), 'sentiment': Value(dtype='string', id=None)}, num_rows: 15000)


In [68]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [69]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.to(device)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [54]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased',model_max_length=128)

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a

In [55]:
def preprocess(data):
  return tokenizer(data['review'], padding=True, truncation=True)

In [56]:
train_set = train_set.map(preprocess, batched=True, batch_size=len(train_set))
test_set = test_set.map(preprocess, batched=True, batch_size=len(test_set))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [57]:
train_set.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_set.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [58]:
train_set

Dataset(features: {'label': Value(dtype='int64', id=None), 'review': Value(dtype='string', id=None), 'sentiment': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}, num_rows: 35000)

In [ ]:
batch_size = 12
epochs = 1

In [60]:
warmup_steps = 500
weight_decay = 0.01

In [61]:
training_args = TrainingArguments(output_dir='./results', num_train_epochs=epochs,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  warmup_steps = warmup_steps,
                                  weight_decay=weight_decay,)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [62]:
from datasets import load_metric

In [63]:
metric = load_metric("accuracy")

In [64]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [65]:
trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=train_set,
                  eval_dataset=test_set,
                  compute_metrics=compute_metrics)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 35000
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2188


Step,Training Loss
500,0.404500
1000,0.339900


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin


In [ ]:
trainer.evaluate()